In [1]:
import pandas as pd
import numpy as np


In [3]:
df_pop = pd.read_csv('../data/curated_data/vic_pop.csv')
df_pop_fore = pd.read_csv('../data/curated_data/population_forecast.csv')
df_post_SA2 = pd.read_csv('../data/raw_data/australian_postcodes.csv')

In [4]:
df_post_SA2 = df_post_SA2[df_post_SA2['state'] == 'VIC']
df_post_SA2 = df_post_SA2.loc[:,['postcode','SA2_MAINCODE_2016']]
df_post_SA2

,postcode,SA2_MAINCODE_2016
6151,3000,206041122.0
6152,3001,206041122.0
6153,3002,206041119.0
6154,3003,206041127.0
6155,3004,206041126.0
...,...,...
18426,8111,206041122.0
18427,8120,206041122.0
18428,8205,206041122.0
18429,8785,212041311.0


In [5]:
df_pop = df_pop.loc[:,['SA2','2021']]

In [6]:
df_pop,df_pop_fore

(           SA2   2021
 0    201011001  16823
 1    201011002  12076
 2    201011003  24348
 3    201011004  25269
 4    201011005   7232
 ..         ...    ...
 457  217031476   3978
 458  217041477   6989
 459  217041478   9963
 460  217041479  22462
 461  217041480  13292
 
 [462 rows x 2 columns],
            SA2 Area Type            Area Name    2026
 0    201011001       SA2            Alfredton  19,227
 1    201011002       SA2             Ballarat  12,534
 2    201011003       SA2     Ballarat - North  25,931
 3    201011004       SA2     Ballarat - South  27,814
 4    201011005       SA2            Buninyong   8,343
 ..         ...       ...                  ...     ...
 457  217031476       SA2                Otway   3,650
 458  217041477       SA2         Moyne - East   6,770
 459  217041478       SA2         Moyne - West  10,612
 460  217041479       SA2  Warrnambool - North  23,959
 461  217041480       SA2  Warrnambool - South  13,577
 
 [462 rows x 4 columns])

In [7]:
df_new_pop = pd.merge(df_pop,df_pop_fore,how='left')

In [8]:
df_new_pop.dropna(axis=0, how='any')

,SA2,2021,Area Type,Area Name,2026
0,201011001,16823,SA2,Alfredton,"19,227"
1,201011002,12076,SA2,Ballarat,"12,534"
2,201011003,24348,SA2,Ballarat - North,"25,931"
3,201011004,25269,SA2,Ballarat - South,"27,814"
4,201011005,7232,SA2,Buninyong,"8,343"
...,...,...,...,...,...
457,217031476,3978,SA2,Otway,"3,650"
458,217041477,6989,SA2,Moyne - East,"6,770"
459,217041478,9963,SA2,Moyne - West,"10,612"
460,217041479,22462,SA2,Warrnambool - North,"23,959"


In [9]:
df_post_SA2['SA2'] = df_post_SA2['SA2_MAINCODE_2016']
df_post_SA2.drop(['SA2_MAINCODE_2016'],axis=1)

,postcode,SA2
6151,3000,206041122.0
6152,3001,206041122.0
6153,3002,206041119.0
6154,3003,206041127.0
6155,3004,206041126.0
...,...,...
18426,8111,206041122.0
18427,8120,206041122.0
18428,8205,206041122.0
18429,8785,212041311.0


In [10]:
df_over = pd.merge(df_post_SA2,df_new_pop,how='left')

In [11]:
df_over

,postcode,SA2_MAINCODE_2016,SA2,2021,Area Type,Area Name,2026
0,3000,206041122.0,206041122.0,43823.0,SA2,Melbourne,"73,594"
1,3001,206041122.0,206041122.0,43823.0,SA2,Melbourne,"73,594"
2,3002,206041119.0,206041119.0,4962.0,SA2,East Melbourne,"6,519"
3,3003,206041127.0,206041127.0,0.0,SA2,West Melbourne,0
4,3004,206041126.0,206041126.0,23173.0,SA2,Southbank,"41,516"
...,...,...,...,...,...,...,...
3529,8111,206041122.0,206041122.0,43823.0,SA2,Melbourne,"73,594"
3530,8120,206041122.0,206041122.0,43823.0,SA2,Melbourne,"73,594"
3531,8205,206041122.0,206041122.0,43823.0,SA2,Melbourne,"73,594"
3532,8785,212041311.0,212041311.0,31330.0,SA2,Dandenong,"40,750"


In [12]:
df_over.dropna(axis=0, how='any')

,postcode,SA2_MAINCODE_2016,SA2,2021,Area Type,Area Name,2026
0,3000,206041122.0,206041122.0,43823.0,SA2,Melbourne,"73,594"
1,3001,206041122.0,206041122.0,43823.0,SA2,Melbourne,"73,594"
2,3002,206041119.0,206041119.0,4962.0,SA2,East Melbourne,"6,519"
3,3003,206041127.0,206041127.0,0.0,SA2,West Melbourne,0
4,3004,206041126.0,206041126.0,23173.0,SA2,Southbank,"41,516"
...,...,...,...,...,...,...,...
3529,8111,206041122.0,206041122.0,43823.0,SA2,Melbourne,"73,594"
3530,8120,206041122.0,206041122.0,43823.0,SA2,Melbourne,"73,594"
3531,8205,206041122.0,206041122.0,43823.0,SA2,Melbourne,"73,594"
3532,8785,212041311.0,212041311.0,31330.0,SA2,Dandenong,"40,750"


In [17]:
df_over = df_over.drop(['SA2_MAINCODE_2016','Area Type'], axis=1)

In [18]:
df_over

,postcode,SA2,2021,Area Name,2026
0,3000,206041122.0,43823.0,Melbourne,"73,594"
1,3001,206041122.0,43823.0,Melbourne,"73,594"
2,3002,206041119.0,4962.0,East Melbourne,"6,519"
3,3003,206041127.0,0.0,West Melbourne,0
4,3004,206041126.0,23173.0,Southbank,"41,516"
...,...,...,...,...,...
3529,8111,206041122.0,43823.0,Melbourne,"73,594"
3530,8120,206041122.0,43823.0,Melbourne,"73,594"
3531,8205,206041122.0,43823.0,Melbourne,"73,594"
3532,8785,212041311.0,31330.0,Dandenong,"40,750"


In [19]:
df_over.to_csv('../data/curated_data/pop_postcode.csv',index = False)

In [25]:
df_pro = pd.read_csv('../data/raw_data/new_properties.csv')

In [33]:
def get_postalcode(name):
    return int(name[-4:])
df_pro['postcode'] = df_pro.name.apply(get_postalcode) 

In [35]:
df_final = pd.merge(df_pro,df_over,how='left')

In [37]:
df_final.to_csv('../data/curated_data/new_properties.csv',index = False)